In [1]:
# datasets: Biblioteca da Hugging Face para carregar e manipular conjuntos de dados
# evaluate: Ferramenta para métricas de avaliação em Machine Learning
# scikit-learn: Biblioteca de aprendizado de máquina com algoritmos e métricas
# transformers: Modelos pré-treinados da Hugging Face para NLP e outras tarefas

!pip install datasets 
!ip install evaluate 
!pip install scikit-learn 
!pip install transformers 

'ip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
# Importa a biblioteca pandas para manipulação de dados tabulares
import pandas as pd

# Importa a função train_test_split do sklearn para dividir os dados em conjuntos de treino e teste
from sklearn.model_selection import train_test_split

# Importa as classes Dataset e DatasetDict da biblioteca datasets para estruturar os dados de forma compatível com a Hugging Face
from datasets import Dataset, DatasetDict

# Importa a classe AutoTokenizer da biblioteca transformers para tokenização automática do texto
from transformers import AutoTokenizer

# Importa a classe DataCollatorWithPadding para garantir padding uniforme nos lotes de dados
from transformers import DataCollatorWithPadding

# Importa a classe AutoModelForSequenceClassification para carregar um modelo pré-treinado para classificação de sequência
# Importa a classe TrainingArguments para definir os parâmetros de treinamento do modelo
# Importa a classe Trainer para facilitar o treinamento do modelo
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Importa as funções accuracy_score e classification_report do sklearn para avaliar o desempenho do modelo
from sklearn.metrics import accuracy_score, classification_report

# Importa a biblioteca evaluate para calcular métricas de avaliação de modelos de Machine Learning
import evaluate

# Importa a biblioteca NumPy para manipulação de arrays numéricos
import numpy as np

# Importa a classe EarlyStoppingCallback da biblioteca transformers para interromper o treinamento
# caso a métrica monitorada não melhore após um número definido de épocas
from transformers import EarlyStoppingCallback
from pytorch_lightning.callbacks import EarlyStopping
from transformers import TrainerCallback

In [3]:
# Dados
data =  pd.read_csv(r"C:\Users\willgnnerferreira\Documents\Modelos\RoBERTaLexPT-base\Dados\dataset_FINAL.csv")

In [ ]:
# Divide o dataset completo em treino (80%) e teste (20%)
# Rodando com 100% dos dados 
train, test = train_test_split(data, 
                               test_size=0.2, 
                               stratify=data["label"], 
                               random_state=42)

print(train.shape, test.shape)

(21424, 3) (5357, 3)


In [5]:
# Cria um objeto DatasetDict (estrutura de dados do Hugging Face) para armazenar os conjuntos de treino e teste
dataset = DatasetDict({
    # Converte o DataFrame de treino para um formato compatível com a biblioteca datasets
    "train": Dataset.from_pandas(train[["text", "label"]], preserve_index=False),
    
    # Converte o DataFrame de teste para um formato compatível com a biblioteca datasets
    "test": Dataset.from_pandas(test[["text", "label"]], preserve_index=False)
})

In [6]:
# eduagarcia/RoBERTaLexPT-base
tokenizer = AutoTokenizer.from_pretrained("eduagarcia/RoBERTaLexPT-base", model_max_length=512)

In [ ]:
def preprocess_function(examples):
    texts = examples["text"]
    # Garantir que é uma lista de strings, para evitar erro, não pode ser formato diferente
    texts = [str(t) for t in texts]
    return tokenizer(texts, truncation=True)

In [8]:
# Remove linhas com valores ausentes em 'text' ou 'label', não pode ter valores nulos
data = data.dropna(subset=["text", "label"]).reset_index(drop=True)

In [9]:
# Aplica a função de pré-processamento ao dataset, processando os dados em lotes
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/21424 [00:00<?, ? examples/s]

Map:   0%|          | 0/5357 [00:00<?, ? examples/s]

In [10]:
# Mostra 5 amostras tokenizadas com o texto original, label e tokens
for i in range(5):
    # Recupera o exemplo original e tokenizado do conjunto de treino
    original_text = dataset["train"][i]["text"]
    label = dataset["train"][i]["label"]
    
    # Tokeniza o texto (usando o mesmo tokenizer)
    tokens = tokenizer.tokenize(original_text)

    print(f"\n--- Amostra {i+1} ---")
    print(f"Texto original: {original_text}")
    print(f"Classe (label): {label}")
    print(f"Tokens: {tokens}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1059 > 512). Running this sequence through the model will result in indexing errors



--- Amostra 1 ---
Texto original: leonel rodrigues de lima lincoln deivid martins oab/go 45.912 oab/go 45.582 l/m advocacia dr. leonel rodrigues de lima: av. abacateiro, qd. 08, lt 12, aeroporto contato: (62) 9 8559-8867 dr. lincoln deivid martins: av. jatai, 141, qd. 50, lt. 19, centro contato: (62) 9 8465-2157 rubiataba/go ao juízo das fazendas públicas da comarca de rubiataba - estado de goiás dalva maria de carvalho, brasileira, solteira, do lar, e-  mail-n/c, inscrita no cpf n° 833.078.181-04 e rg 360022 dgpc, é residente e domiciliada na rua lírio, qd. 17, lt. 14, residencial eldorado, rubiataba – goiás, cep 76.350-000, por intermédio de seus advogados e bastantes procuradores (m.j.), com escritório profissional indicado no rodapé desta exordial, vem à presença de vossa excelência, pleitear: pensão por morte urbana em face do instituto nacional do seguro social - inss autarquia federal inscrita no cnpj n° 29.979.036/0001-40, com agência na localizada na rua quinze de dezembro, n

In [11]:
# Cria um Data Collator para adicionar padding automaticamente aos inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
# Carrega a métrica de acurácia da biblioteca evaluate
accuracy = evaluate.load("accuracy")

In [13]:
# Função para calcular métricas de avaliação do modelo
def compute_metrics(eval_pred):
    # Desempacota as previsões e os rótulos verdadeiros (labels)
    predictions, labels = eval_pred

    # Obtém o índice da classe com maior probabilidade para cada previsão
    predictions = np.argmax(predictions, axis=1)

    # Calcula a acurácia comparando as previsões com os rótulos reais
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
# Dicionário que mapeia os IDs das classes para seus respectivos rótulos
id2label = {
    1: "FATO",   # Classe 1 representa "FATO"
    0: "TESE",   # Classe 0 representa "TESE"
    2: "RUIDO"   # Classe 2 representa "RUIDO"
}

# Dicionário inverso que mapeia os rótulos das classes para seus respectivos IDs numéricos
label2id = {
    "FATO": 1,   # "FATO" é representado pelo ID 1
    "TESE": 0,   # "TESE" é representado pelo ID 0
    "RUIDO": 2   # "RUIDO" é representado pelo ID 2
}

In [15]:
# Mostrar algumas amostras do dataset com rótulos traduzidos
for i in range(10):  # Exibe 10 exemplos
    text = dataset["train"][i]["text"]
    label_id = dataset["train"][i]["label"]
    label_name = id2label.get(label_id, "Desconhecido")  # Converte ID para nome
    
    print(f"Texto: {text}\nLabel ID: {label_id} -> Classe: {label_name}\n{'-'*50}")

Texto: leonel rodrigues de lima lincoln deivid martins oab/go 45.912 oab/go 45.582 l/m advocacia dr. leonel rodrigues de lima: av. abacateiro, qd. 08, lt 12, aeroporto contato: (62) 9 8559-8867 dr. lincoln deivid martins: av. jatai, 141, qd. 50, lt. 19, centro contato: (62) 9 8465-2157 rubiataba/go ao juízo das fazendas públicas da comarca de rubiataba - estado de goiás dalva maria de carvalho, brasileira, solteira, do lar, e-  mail-n/c, inscrita no cpf n° 833.078.181-04 e rg 360022 dgpc, é residente e domiciliada na rua lírio, qd. 17, lt. 14, residencial eldorado, rubiataba – goiás, cep 76.350-000, por intermédio de seus advogados e bastantes procuradores (m.j.), com escritório profissional indicado no rodapé desta exordial, vem à presença de vossa excelência, pleitear: pensão por morte urbana em face do instituto nacional do seguro social - inss autarquia federal inscrita no cnpj n° 29.979.036/0001-40, com agência na localizada na rua quinze de dezembro, nº 249, setor central, cep 75

In [16]:
# Carrega o modelo pré-treinado BERT para classificação de sequência
model = AutoModelForSequenceClassification.from_pretrained(
    "eduagarcia/RoBERTaLexPT-base",  # Nome do modelo pré-treinado em português
    num_labels=3,  # Número de classes na classificação (FATO, TESE, RUIDO)
    id2label=id2label,  # Dicionário que mapeia IDs para nomes das classes
    label2id=label2id   # Dicionário que mapeia nomes das classes para IDs
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at eduagarcia/RoBERTaLexPT-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Define os parâmetros do treinamento do modelo
training_args = TrainingArguments(
    output_dir="model_128tokens", # Diretório onde os modelos treinados serão salvos
    learning_rate=2e-5, # Taxa de aprendizado para otimização do modelo
    per_device_train_batch_size=32, # Tamanho do batch de treinamento por GPU/CPU
    per_device_eval_batch_size=32, # Tamanho do batch de avaliação por GPU/CPU
    num_train_epochs=10,  # Número de épocas de treinamento (quantas vezes o modelo verá todo o conjunto de dados)
    weight_decay=0.01, # Decaimento de peso para regularização e evitar overfitting
    eval_strategy="epoch", # Avaliação será feita ao final de cada época
    save_strategy="epoch", # Salvará o modelo ao final de cada época
    load_best_model_at_end=True, # Carrega o melhor modelo baseado na métrica de avaliação ao final do treinamento
    metric_for_best_model="accuracy", # Define que o melhor modelo será selecionado com base na maior acurácia obtida durante as avaliações.
    save_total_limit=2, # Mantém apenas os dois checkpoints mais recentes (ou melhores) no disco, economizando espaço de armazenamento.
    # push_to_hub=True,  # (Comentado) Caso ativado, envia o modelo para o Hugging Face Hub automaticamente
)

# Criação do Trainer, responsável pelo treinamento do modelo
trainer = Trainer(
    model=model,  # Modelo carregado anteriormente
    args=training_args,  # Argumentos de treinamento definidos acima
    train_dataset=tokenized_dataset["train"],  # Conjunto de treinamento tokenizado
    eval_dataset=tokenized_dataset["test"],  # Conjunto de teste tokenizado
    processing_class=tokenizer,  # Define o tokenizador para ser usado pelo Trainer 
    data_collator=data_collator,  # Gerencia o padding dos inputs para que fiquem no mesmo tamanho
    compute_metrics=compute_metrics,  # Função que calcula as métricas de avaliação do modelo
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] # Para o treinamnto após 2 épocas, se não tiver melhora
)

# Inicia o treinamento do modelo
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.147900,0.065212,0.985440
2,0.058400,0.039958,0.988240
3,0.028700,0.048805,0.989360
4,0.015200,0.049151,0.990106
5,0.008900,0.060178,0.990853
6,0.005800,0.061567,0.989360
7,0.002800,0.064506,0.990293


TrainOutput(global_step=4690, training_loss=0.033373150648847064, metrics={'train_runtime': 161256.9582, 'train_samples_per_second': 1.329, 'train_steps_per_second': 0.042, 'total_flos': 3.94585930298327e+16, 'train_loss': 0.033373150648847064, 'epoch': 7.0})

In [18]:
# Obtém as previsões do modelo no conjunto de teste
predictions = trainer.predict(tokenized_dataset["test"])

# Extrai os logits (saída bruta do modelo antes da ativação softmax)
logits = predictions.predictions

# Converte os logits para rótulos preditos (classe com maior probabilidade)
preds = np.argmax(logits, axis=1)

# Converte os rótulos reais para numpy array
labels = np.array(tokenized_dataset["test"]['label'])

# Calcula a acurácia
accuracy = accuracy_score(labels, preds)

# Define os nomes das classes a partir do dicionário label2id
label_names = list(label2id.keys())

# Exibe os resultados
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(labels, preds, target_names=label_names))

Accuracy: 0.9909
              precision    recall  f1-score   support

        FATO       0.99      0.99      0.99      1785
        TESE       0.99      0.99      0.99      1786
       RUIDO       1.00      1.00      1.00      1786

    accuracy                           0.99      5357
   macro avg       0.99      0.99      0.99      5357
weighted avg       0.99      0.99      0.99      5357

